# A: Thu thập dữ liệu cách parse HTML
---

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
import os
import csv
import pickle
from datetime import datetime, timedelta
import concurrent.futures

---

## Setup ở mức toàn cục

Các setup cần thiết: 
- Cần kiểm tra đã có thư mục Crawl_data hay chưa nếu chưa thì ta cần tạo thư mục mới.
- Khi parse HTML, chúng ta sẽ cho sleep giữa các lần thực hiện request do đó set biến sleep_time = 1.
- Thiết lập cache để giảm số lần thu thập dữ liệu: thời gian hết hạn là None.

In [2]:
if not os.path.exists('Crawl_data/'):
    os.makedirs('Crawl_data/')

In [3]:
sleep_time = 1

In [4]:
requests_cache.install_cache(expire_after=None)

Ngoài ra, em setup những cài đặt cần thiết cho driver selenium cho toàn bộ bài làm này với 1 số thuộc tính cơ bản, trong đó có phần `headless` nghĩa là sẽ chạy selenium dưới dạng ngầm định và giúp giảm thời gian chạy của bài làm

In [5]:
option = Options()

option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
option.add_argument("--headless")
option.add_experimental_option("excludeSwitches", ['enable-automation']);

# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2,
    "profile.managed_default_content_settings.images": 2
})

---

## Quy trình thực hiện bài làm:
- Bước 1: Thu thập tất cả các thể loại music hiện có trên soundcloud (30 thể loại music)
- Bước 2: Tạo ra các link tìm kiếm từ các thể loại nhạc thu thập được để tìm kiếm các playlist liên quan thuộc thể loại nhạc đó. (30 link tìm kiếm)
- Bước 3: Với mỗi link ở B2, lấy khoảng 50 link `playlist` của thể loại đó và lưu tất cả chúng vào biến `playlist_link` có kiểu dữ liệu là set (tránh trùng lặp playlist)
- Bước 4: Với mỗi playlist_link, lấy ra tên `user` của người tạo ra playlist và lưu vào biến `user_link` có kiểu dữ liệu là set (tránh trùng lặp user)
- Bước 5: Với mỗi playlist_link, lấy ra link của 2 `track` đầu tiên và lưu vào biến `track_link` có kiểu dữ liệu là set (tránh trùng lặp track)

Sau khi hoàn thành bước 5, lúc này chúng ta có 3 biến là `playlist_link`, `user_link` và `user_link` chứa đường link tới các trang cần thu thập dữ liệu.
- Bước 6: Thu thập dữ liệu `playlist` bằng cách truy cập vào các link trong `playlist_link` và lưu vào file
- Bước 7: Thu thập dữ liệu `user` bằng cách truy cập vào các link trong `user_link` và lưu vào file
- Bước 8: Thu thập dữ liệu `track` bằng cách truy cập vào các link trong `track_link` và lưu vào file

## Một số lưu ý khi chạy file:
- Đối với bài làm của em mỗi khi chúng ta `Restart - Run all` thì sẽ cho ra 1 kết quả khác nhau, lí do là vì em làm bài theo hướng universal do đó mỗi khi dùng link để tìm kiếm trên soundcloud thì sẽ thu lại được kết quả khác nhau.
- Những dòng markdown em ghi bên dưới có số liệu kèm theo có thể sẽ khác với lúc thầy chạy bài.

---

## Bước 1: Thu thập tất cả các thể loại music hiện có trên soundcloud (30 thể loại music)

Chúng em đã thu thập được 30 thể loại music hiện có trên soundcloud và ghi chúng vào file `genre_file.txt` và giờ đọc lại file để có list những genre nhạc.

In [6]:
genre_list = []
with open('genre_file.txt', 'r', encoding="utf-8") as fin:
        data = fin.read().strip()
        genre_list = data.split('\n')

In [7]:
genre_list

['Alternative Rock',
 'Ambient',
 'Classical',
 'Country',
 'Dance & EDM',
 'Dancehall',
 'Deep House',
 'Disco',
 'Drum & Bass',
 'Dubstep',
 'Electronic',
 'Folk & Singer-Songwriter',
 'Hip-hop & Rap',
 'House',
 'Indie',
 'Jazz & Blues',
 'Latin',
 'Metal',
 'Piano',
 'Pop',
 'R&B & Soul',
 'Reggae',
 'Reggaeton',
 'Rock',
 'Soundtrack',
 'Techno',
 'Trance',
 'Trap',
 'Triphop',
 'World']

---

## Bước 2: Tạo ra các link tìm kiếm từ các thể loại nhạc thu thập được để tìm kiếm các playlist liên quan thuộc thể loại nhạc đó. (30 link tìm kiếm)

Từ list các genre nhạc, em sẽ tạo ra link tìm kiếm có dạng `f'https://soundcloud.com/search/sets?q={genre}&filter.genre_or_tag={genre}'` dạng f-string để tạo ra các link tìm kiếm. Ở đây `https://soundcloud.com/search/sets?q=` là đường dẫn tiền tố giúp chúng ta tìm được các playlist, `genre` là thể loại nhạc, `&filter.genre_or_tag` là tag giúp chúng ta tìm theo thể chính thể loại nhạc đó. Em chuẩn hóa chuỗi bằng cách thay `" "` thành `"%20"` và `"&"` thành `"%26"` để dể dàng tạo ra link.

In [8]:
search_url = []
for genre in genre_list:
    genre = genre.replace(" ", "%20")
    genre = genre.replace("&", "%26")
    url = f'https://soundcloud.com/search/sets?q={genre}&filter.genre_or_tag={genre}'
    search_url.append(url)

In [9]:
search_url

['https://soundcloud.com/search/sets?q=Alternative%20Rock&filter.genre_or_tag=Alternative%20Rock',
 'https://soundcloud.com/search/sets?q=Ambient&filter.genre_or_tag=Ambient',
 'https://soundcloud.com/search/sets?q=Classical&filter.genre_or_tag=Classical',
 'https://soundcloud.com/search/sets?q=Country&filter.genre_or_tag=Country',
 'https://soundcloud.com/search/sets?q=Dance%20%26%20EDM&filter.genre_or_tag=Dance%20%26%20EDM',
 'https://soundcloud.com/search/sets?q=Dancehall&filter.genre_or_tag=Dancehall',
 'https://soundcloud.com/search/sets?q=Deep%20House&filter.genre_or_tag=Deep%20House',
 'https://soundcloud.com/search/sets?q=Disco&filter.genre_or_tag=Disco',
 'https://soundcloud.com/search/sets?q=Drum%20%26%20Bass&filter.genre_or_tag=Drum%20%26%20Bass',
 'https://soundcloud.com/search/sets?q=Dubstep&filter.genre_or_tag=Dubstep',
 'https://soundcloud.com/search/sets?q=Electronic&filter.genre_or_tag=Electronic',
 'https://soundcloud.com/search/sets?q=Folk%20%26%20Singer-Songwriter&f

---

## Bước 3: Với mỗi link ở B2, lấy khoảng 50 link playlist của thể loại đó và lưu tất cả chúng vào biến playlist_link có kiểu dữ liệu là set (tránh trùng lặp playlist)

Ở bước này, em sẽ dùng selenium để truy cập vào 30 đường link tìm kiếm đã tạo được từ B2, tuy nhiên nếu chỉ đơn thuần gọi 1 driver selenium và chạy nó để truy cập 30 link và thu thập 50 playlist sẽ khá lâu và tốn thời gian, do đó em áp dụng `concurrent.futures` để có thể chạy cùng lúc là 5 driver selenium và mỗi driver sẽ chịu trách nhiệm tìm kiếm cho 6 link giúp giảm bớt thời gian thực thi đoạn code. (Chạy đa luồng)

Đầu tiên, em viết hàm `get_playlist_link`. Hàm này có các input:
- `search_url`: chính là danh sách các link tìm kiếm đã có được ở B2.
- `start`: vị trí bắt đầu của url cần làm việc khi chạy từng driver selenium.

In [10]:
def get_playlist_link(search_url, start):
    
    driver = webdriver.Chrome(options=option, executable_path='chromedriver')
    
    for url in search_url[start:start+6]:
        driver.get(url)
        time.sleep(sleep_time)
        playlist = []
        while len(playlist) < 50:
            playlist = (driver.find_elements(By.CSS_SELECTOR, "li[class='searchList__item']"))
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(1)
        for ele in playlist:
            link = ele.find_element(By.CSS_SELECTOR, "a[class='sound__coverArt']")
            link = link.get_attribute('href')
            playlist_link.add(link)
    
    driver.quit()

- Sử dụng đa luồng để thực hiện

In [11]:
playlist_link = set()
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_playlist_link, search_url, start) for start in range(0, 30, 6)}

In [12]:
# TEST
assert len(playlist_link) > 1000
len(playlist_link)

1490

Như vậy sau khi thực hiện xong, em thu thập được `playlist_link` có độ dài là **1490** playlist, hoàn thành bước **3**.

---

## Bước 4: Với mỗi playlist_link, lấy ra tên user của người tạo ra playlist và lưu vào biến user_link có kiểu dữ liệu là set (tránh trùng lặp user)

In [13]:
def create_user_link_list(playlist_link):
    user_link = set()
    for link in playlist_link:
        flag = link.find("/sets/")
        user_link.add(link[0:flag])
    return user_link

In [14]:
user_link = create_user_link_list(playlist_link)

In [15]:
# TEST
assert len(user_link) > 1000
len(user_link)

1354

Như vậy sau khi thực hiện xong, em thu thập được `user_link` có độ dài là **1354** user, hoàn thành bước **4**.

---

## Bước 5: Với mỗi playlist_link, lấy ra link của 2 track đầu tiên và lưu vào biến track_link có kiểu dữ liệu là set (tránh trùng lặp track)

Tương tự như B3, ở bước này em tiếp tục dùng `concurrent.futures` để chạy đa luồng nhằm tăng tốc độ thu thập dữ liệu.

Đầu tiên, em viết hàm `create_track_link_list`. Hàm này có các input:
- `playlist_link`: chính là danh sách các playlist_link tìm được ở B3.
- `pre_url`: là chuỗi `https://soundcloud.com`
- `start`: vị trí bắt đầu của `playlist_link` khi chạy 1 luồng mới
- `step`: bước nhảy, ở đây step bằng int(len(playlist_link) / 10) vì em muốn chạy 10 luồng

In [16]:
def create_track_link_list(playlist_link, pre_url, start, step):
    
    playlist_link = list(playlist_link)
    for url in playlist_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        tracks = soup.select('article[itemprop=track]')[:2]
        for track in tracks:
            link = track.find("a")["href"]
            track_link.add(pre_url + link)

- Sử dụng đa luồng để thực hiện

In [17]:
track_link = set()
step = int(len(playlist_link) / 10)
pre_url = "https://soundcloud.com"
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(create_track_link_list, playlist_link, pre_url, start, step) for start in range(0, len(playlist_link), step)}

In [18]:
# TEST
assert len(track_link) > 1000
len(track_link)

2863

Như vậy sau khi thực hiện xong, em thu thập được `track_link` có độ dài là **2863** track, hoàn thành bước **5**.

---

## Bước 6: Thu thập dữ liệu playlist bằng cách truy cập vào các link trong file playlist_link

Hàm `get_info_playlist` có các thành phần:
- `playlist_link`: chứa các link playlist
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [19]:
def get_info_playlist(playlist_link, start, step):
    playlist_link = list(playlist_link)
    for url in playlist_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        
        # tìm kiếm script có chứa đoạn text: window.__sc_hydration
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
                    
        pos = temp.find('{"artwork_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        
        data.pop("user")  # vì trong data còn có key user lưu trữ thông tin của người đăng playlist do đó loại bỏ nó ra
        
        # lấy các track ids có trong playlist và tạo chuỗi theo yêu cầu đề bài
        tracks = data.pop("tracks")
        tracksid = ""
        for track in tracks:
            tracksid += str(track["id"]) + ","
        tracksid = tracksid.rstrip(',')
        data["TracksId"] = tracksid
        playlist_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [20]:
playlist_lst = []
step = int(len(playlist_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_playlist, playlist_link, start, step) for start in range(0, len(playlist_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các playlist dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `playlist_lst` thành 1 Data Frame của `pandas`
- `df_pl` là biến lưu trữ cho `playlist_lst`

In [184]:
df_pl = pd.json_normalize(playlist_lst)
df_pl.columns

Index(['artwork_url', 'created_at', 'description', 'duration', 'embeddable_by',
       'genre', 'id', 'kind', 'label_name', 'last_modified', 'license',
       'likes_count', 'managed_by_feeds', 'permalink', 'permalink_url',
       'public', 'purchase_title', 'purchase_url', 'release_date',
       'reposts_count', 'secret_token', 'sharing', 'tag_list', 'title', 'uri',
       'user_id', 'set_type', 'is_album', 'published_at', 'display_date',
       'track_count', 'url', 'TracksId'],
      dtype='object')

###  Theo dõi 5 dòng đầu của `df_pl`

In [185]:
df_pl.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,None,2017-09-15T02:41:19Z,None,114356108,all,Reggae,352781337,playlist,None,2020-04-04T16:19:58Z,...,Dancehall,https://api.soundcloud.com/playlists/352781337,197755511,,False,2017-09-15T02:41:19Z,2017-09-15T02:41:19Z,448,/user-818098543/sets/dancehall,"5963010,163143507,176991526,115524663,17063332..."
1,https://i1.sndcdn.com/artworks-XAH1mvlUyhfDT70...,2019-08-09T15:27:57Z,,10917613,all,Chill EDM,843427727,playlist,,2021-11-12T14:36:50Z,...,Chill EDM: Waves,https://api.soundcloud.com/playlists/843427727,603452106,,False,2019-08-09T20:00:44Z,2019-08-09T20:00:44Z,53,/soundcloud-the-peak/sets/waves-chill-edm,"1153337647,1142050318,1138569328,1146058903,11..."
2,None,2014-12-31T06:29:43Z,None,150327799,none,Electronic,68396635,playlist,None,2017-01-20T03:05:06Z,...,Dubstep,https://api.soundcloud.com/playlists/68396635,109662100,,False,None,2014-12-31T06:29:43Z,456,/damogaur86/sets/dubstep,"130054263,154791745,171835907,26549785,1608792..."
3,None,2020-05-16T20:37:26Z,Pop Smoke,5554371,all,Hip-hop & Rap,1052484982,playlist,,2021-10-28T02:24:01Z,...,Pop,https://api.soundcloud.com/playlists/1052484982,517254165,,False,2020-05-16T20:37:26Z,2020-05-16T20:37:26Z,33,/pop-smoke-12345/sets/pop-smoke,"656255261,610276575,752519617,687445048,752519..."
4,None,2018-06-07T22:39:43Z,None,231259392,all,Electronic,536039613,playlist,None,2018-10-01T14:49:34Z,...,Electronic,https://api.soundcloud.com/playlists/536039613,403437198,,False,2018-06-12T00:16:07Z,2018-06-12T00:16:07Z,476,/user-733308340-928877766/sets/electronic-scamp,"304971267,124259977,168029313,174500966,107547..."


### Ghi dữ liệu ra file `playlist.csv`

In [186]:
playlist_file = "Crawl_data/playlist.csv"
df_pl.to_csv(sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `playlist.csv`

In [187]:
# TEST
playlists = pd.read_csv(playlist_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về playlist chưa
assert len(playlists) > 1000

# Xem 5 dòng đầu của dữ liệu
playlists.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,NaN,2017-09-15T02:41:19Z,NaN,114356108,all,Reggae,352781337,playlist,NaN,2020-04-04T16:19:58Z,...,Dancehall,https://api.soundcloud.com/playlists/352781337,197755511,NaN,False,2017-09-15T02:41:19Z,2017-09-15T02:41:19Z,448,/user-818098543/sets/dancehall,"5963010,163143507,176991526,115524663,17063332..."
1,https://i1.sndcdn.com/artworks-XAH1mvlUyhfDT70...,2019-08-09T15:27:57Z,NaN,10917613,all,Chill EDM,843427727,playlist,NaN,2021-11-12T14:36:50Z,...,Chill EDM: Waves,https://api.soundcloud.com/playlists/843427727,603452106,NaN,False,2019-08-09T20:00:44Z,2019-08-09T20:00:44Z,53,/soundcloud-the-peak/sets/waves-chill-edm,"1153337647,1142050318,1138569328,1146058903,11..."
2,NaN,2014-12-31T06:29:43Z,NaN,150327799,none,Electronic,68396635,playlist,NaN,2017-01-20T03:05:06Z,...,Dubstep,https://api.soundcloud.com/playlists/68396635,109662100,NaN,False,NaN,2014-12-31T06:29:43Z,456,/damogaur86/sets/dubstep,"130054263,154791745,171835907,26549785,1608792..."
3,NaN,2020-05-16T20:37:26Z,Pop Smoke,5554371,all,Hip-hop & Rap,1052484982,playlist,NaN,2021-10-28T02:24:01Z,...,Pop,https://api.soundcloud.com/playlists/1052484982,517254165,NaN,False,2020-05-16T20:37:26Z,2020-05-16T20:37:26Z,33,/pop-smoke-12345/sets/pop-smoke,"656255261,610276575,752519617,687445048,752519..."
4,NaN,2018-06-07T22:39:43Z,NaN,231259392,all,Electronic,536039613,playlist,NaN,2018-10-01T14:49:34Z,...,Electronic,https://api.soundcloud.com/playlists/536039613,403437198,NaN,False,2018-06-12T00:16:07Z,2018-06-12T00:16:07Z,476,/user-733308340-928877766/sets/electronic-scamp,"304971267,124259977,168029313,174500966,107547..."


---

## Bước 7: Thu thập dữ liệu user bằng cách truy cập vào các link trong file user_link

Hàm `get_info_user` có các thành phần:
- `user_link`: chứa các link user
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [28]:
def get_info_user(user_link, start, step):
    user_link = list(user_link)
    for url in user_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
        pos = temp.find('{"avatar_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        user_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [ ]:
user_lst = []
step = int(len(user_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_user, user_link, start, step) for start in range(0, len(user_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các user dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `user_lst` thành 1 Data Frame của `pandas`
- `df_us` là biến lưu trữ cho `user_lst`

In [188]:
df_us = pd.json_normalize(user_lst)
df_us.columns

Index(['avatar_url', 'city', 'comments_count', 'country_code', 'created_at',
       'creator_subscriptions', 'description', 'followers_count',
       'followings_count', 'first_name', 'full_name', 'groups_count', 'id',
       'kind', 'last_modified', 'last_name', 'likes_count',
       'playlist_likes_count', 'permalink', 'permalink_url', 'playlist_count',
       'reposts_count', 'track_count', 'uri', 'urn', 'username', 'verified',
       'visuals', 'station_urn', 'station_permalink', 'url',
       'creator_subscription.product.id', 'badges.pro', 'badges.pro_unlimited',
       'badges.verified', 'visuals.urn', 'visuals.enabled', 'visuals.visuals',
       'visuals.tracking'],
      dtype='object')

###  Theo dõi 5 dòng đầu của `df_us`

In [189]:
df_us.head()

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,first_name,...,station_permalink,url,creator_subscription.product.id,badges.pro,badges.pro_unlimited,badges.verified,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,https://i1.sndcdn.com/avatars-000055961532-om0...,,86,None,2013-07-29T05:25:14Z,[{'product': {'id': 'free'}}],None,1411,1741,ʚïɞSue,...,artist-stations:52557367,/saratology,free,False,False,False,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/avatars-000435801174-dzp...,Los Angeles,0,US,2013-03-05T17:51:51Z,[{'product': {'id': 'creator-pro-unlimited'}}],None,4964,2,,...,artist-stations:38296485,/backlotmusic,creator-pro-unlimited,False,True,False,soundcloud:users:38296485,True,"[{'urn': 'soundcloud:visuals:111760284', 'entr...",NaN
2,https://i1.sndcdn.com/avatars-IMLvkSzRUua5FWaU...,None,2,None,2018-04-16T16:18:54Z,[{'product': {'id': 'free'}}],None,8,40,SATIVA,...,artist-stations:434139399,/sativa-mitchell-21088,free,False,False,False,soundcloud:users:434139399,True,"[{'urn': 'soundcloud:visuals:46593301', 'entry...",NaN
3,https://i1.sndcdn.com/avatars-000220689128-a2w...,None,6,None,2013-09-09T03:12:12Z,[{'product': {'id': 'free'}}],None,81,119,,...,artist-stations:57382585,/carocj,free,False,False,False,soundcloud:users:57382585,True,"[{'urn': 'soundcloud:visuals:9268568', 'entry_...",NaN
4,https://i1.sndcdn.com/avatars-jTAoMxJjdFXggopi...,reading studying zen,0,None,2020-04-13T13:57:47Z,[{'product': {'id': 'free'}}],None,785,9,for meditation epic chill workout,...,artist-stations:809060917,/music-for-life-playlists,free,False,False,False,NaN,NaN,NaN,NaN


### Ghi dữ liệu ra file `user.csv`

In [220]:
user_file = "Crawl_data/user.csv"
df_us.to_csv(user_file, sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `user.csv`

In [221]:
# TEST
users = pd.read_csv(user_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về user chưa
assert len(users) > 1000

# Xem 5 dòng đầu của dữ liệu
users.head()

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,first_name,...,station_permalink,url,creator_subscription.product.id,badges.pro,badges.pro_unlimited,badges.verified,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,https://i1.sndcdn.com/avatars-000055961532-om0...,NaN,86,NaN,2013-07-29T05:25:14Z,[{'product': {'id': 'free'}}],NaN,1411,1741,ʚïɞSue,...,artist-stations:52557367,/saratology,free,False,False,False,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/avatars-000435801174-dzp...,Los Angeles,0,US,2013-03-05T17:51:51Z,[{'product': {'id': 'creator-pro-unlimited'}}],NaN,4964,2,NaN,...,artist-stations:38296485,/backlotmusic,creator-pro-unlimited,False,True,False,soundcloud:users:38296485,True,"[{'urn': 'soundcloud:visuals:111760284', 'entr...",NaN
2,https://i1.sndcdn.com/avatars-IMLvkSzRUua5FWaU...,NaN,2,NaN,2018-04-16T16:18:54Z,[{'product': {'id': 'free'}}],NaN,8,40,SATIVA,...,artist-stations:434139399,/sativa-mitchell-21088,free,False,False,False,soundcloud:users:434139399,True,"[{'urn': 'soundcloud:visuals:46593301', 'entry...",NaN
3,https://i1.sndcdn.com/avatars-000220689128-a2w...,NaN,6,NaN,2013-09-09T03:12:12Z,[{'product': {'id': 'free'}}],NaN,81,119,NaN,...,artist-stations:57382585,/carocj,free,False,False,False,soundcloud:users:57382585,True,"[{'urn': 'soundcloud:visuals:9268568', 'entry_...",NaN
4,https://i1.sndcdn.com/avatars-jTAoMxJjdFXggopi...,reading studying zen,0,NaN,2020-04-13T13:57:47Z,[{'product': {'id': 'free'}}],NaN,785,9,for meditation epic chill workout,...,artist-stations:809060917,/music-for-life-playlists,free,False,False,False,NaN,NaN,NaN,NaN


---

## Bước 8: Thu thập dữ liệu track bằng cách truy cập vào các link trong file track_link

Hàm `get_info_track` có các thành phần:
- `track_link`: chứa các link track
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [149]:
def get_info_track(track_link, start, step):
    track_link = list(track_link)
    for url in track_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
        pos = temp.find('{"artwork_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        data.pop('user')
        track_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [150]:
track_lst = []
step = int(len(track_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_track, track_link, start, step) for start in range(0, len(track_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các track dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `track_lst` thành 1 Data Frame của `pandas`
- `df_tr` là biến lưu trữ cho `track_lst`

In [192]:
df_tr = pd.json_normalize(track_lst)
df_tr.columns

Index(['artwork_url', 'caption', 'commentable', 'comment_count', 'created_at',
       'description', 'downloadable', 'download_count', 'duration',
       'full_duration', 'embeddable_by', 'genre', 'has_downloads_left', 'id',
       'kind', 'label_name', 'last_modified', 'license', 'likes_count',
       'permalink', 'permalink_url', 'playback_count', 'public',
       'publisher_metadata', 'purchase_title', 'purchase_url', 'release_date',
       'reposts_count', 'secret_token', 'sharing', 'state', 'streamable',
       'tag_list', 'title', 'track_format', 'uri', 'urn', 'user_id', 'visuals',
       'waveform_url', 'display_date', 'station_urn', 'station_permalink',
       'track_authorization', 'monetization_model', 'policy',
       'media.transcodings', 'publisher_metadata.id', 'publisher_metadata.urn',
       'publisher_metadata.artist', 'publisher_metadata.album_title',
       'publisher_metadata.contains_music', 'publisher_metadata.upc_or_ean',
       'publisher_metadata.isrc', 'publis

###  Theo dõi 5 dòng đầu của `df_tr`

In [193]:
df_tr.head()

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,...,publisher_metadata.c_line,publisher_metadata.c_line_for_display,publisher_metadata.release_title,publisher_metadata.writer_composer,publisher_metadata.publisher,publisher_metadata.iswc,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,None,None,True,1.0,2012-11-27T19:02:04Z,,False,0.0,420453,420453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000045729697-23...,None,True,33.0,2013-04-17T13:32:12Z,Artist: Vedomir (Marcel Dettmann remixes)\r\nF...,False,0.0,248943,248943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://i1.sndcdn.com/artworks-000149186638-zs...,None,True,35.0,2016-03-01T05:13:23Z,Fresh off of the buzz from his most recent re...,False,0.0,189509,189509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://i1.sndcdn.com/artworks-KOQjX4KNCzmQ-0-...,None,True,10.0,2016-04-01T19:36:01Z,None,False,0.0,233430,233430,...,"Â© 2006 Gasoline Alley, J.V.","Â© 2006 Gasoline Alley, J.V.","April 29, 1992 (Miami)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://i1.sndcdn.com/artworks-vA1Gvw7Rqbdi-0-...,None,True,5.0,2021-11-03T23:36:01Z,None,False,0.0,30000,250331,...,"An Atlantic Records UK release, Â© 2021 Warner...","Â© An Atlantic Records UK release, Â© 2021 War...",Faisal (Envelops Me),"Benjy Gibson, Emmanuel Franklyn Adelabu, Fasia...",NaN,NaN,NaN,NaN,NaN,NaN


### Ghi dữ liệu ra file `track.csv`

In [194]:
track_file = "Crawl_data/track.csv"
df_tr.to_csv(track_file, sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `track.csv`

In [195]:
# TEST
tracks = pd.read_csv(track_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về track chưa
assert len(tracks) > 1000

# Xem 5 dòng đầu của dữ liệu
tracks.head()

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,...,publisher_metadata.c_line,publisher_metadata.c_line_for_display,publisher_metadata.release_title,publisher_metadata.writer_composer,publisher_metadata.publisher,publisher_metadata.iswc,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,NaN,NaN,True,1.0,2012-11-27T19:02:04Z,NaN,False,0.0,420453,420453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000045729697-23...,NaN,True,33.0,2013-04-17T13:32:12Z,Artist: Vedomir (Marcel Dettmann remixes)\r\nF...,False,0.0,248943,248943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://i1.sndcdn.com/artworks-000149186638-zs...,NaN,True,35.0,2016-03-01T05:13:23Z,Fresh off of the buzz from his most recent re...,False,0.0,189509,189509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://i1.sndcdn.com/artworks-KOQjX4KNCzmQ-0-...,NaN,True,10.0,2016-04-01T19:36:01Z,NaN,False,0.0,233430,233430,...,"Â© 2006 Gasoline Alley, J.V.","Â© 2006 Gasoline Alley, J.V.","April 29, 1992 (Miami)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://i1.sndcdn.com/artworks-vA1Gvw7Rqbdi-0-...,NaN,True,5.0,2021-11-03T23:36:01Z,NaN,False,0.0,30000,250331,...,"An Atlantic Records UK release, Â© 2021 Warner...","Â© An Atlantic Records UK release, Â© 2021 War...",Faisal (Envelops Me),"Benjy Gibson, Emmanuel Franklyn Adelabu, Fasia...",NaN,NaN,NaN,NaN,NaN,NaN


---

## Phân tích và xử lí dữ liệu thu được

### **1) Phân tích đối với tập playlist:**

Sử dụng lại biến `playlists` (Data Frame) đã khai báo ở B6.

In [201]:
print(f'Số dòng và số cột dữ liệu của file playlist.csv {playlists.shape}')
print(f'Số dòng dữ liệu {playlists.shape[0]}')
print(f'Số cột dữ liệu {playlists.shape[1]}')

Số dòng và số cột dữ liệu của file playlist.csv (1490, 33)
Số dòng dữ liệu 1490
Số cột dữ liệu 33


In [203]:
# Xem thông tin 5 dòng đầu tiên
playlists.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,NaN,2017-09-15T02:41:19Z,NaN,114356108,all,Reggae,352781337,playlist,NaN,2020-04-04T16:19:58Z,...,Dancehall,https://api.soundcloud.com/playlists/352781337,197755511,NaN,False,2017-09-15T02:41:19Z,2017-09-15T02:41:19Z,448,/user-818098543/sets/dancehall,"5963010,163143507,176991526,115524663,17063332..."
1,https://i1.sndcdn.com/artworks-XAH1mvlUyhfDT70...,2019-08-09T15:27:57Z,NaN,10917613,all,Chill EDM,843427727,playlist,NaN,2021-11-12T14:36:50Z,...,Chill EDM: Waves,https://api.soundcloud.com/playlists/843427727,603452106,NaN,False,2019-08-09T20:00:44Z,2019-08-09T20:00:44Z,53,/soundcloud-the-peak/sets/waves-chill-edm,"1153337647,1142050318,1138569328,1146058903,11..."
2,NaN,2014-12-31T06:29:43Z,NaN,150327799,none,Electronic,68396635,playlist,NaN,2017-01-20T03:05:06Z,...,Dubstep,https://api.soundcloud.com/playlists/68396635,109662100,NaN,False,NaN,2014-12-31T06:29:43Z,456,/damogaur86/sets/dubstep,"130054263,154791745,171835907,26549785,1608792..."
3,NaN,2020-05-16T20:37:26Z,Pop Smoke,5554371,all,Hip-hop & Rap,1052484982,playlist,NaN,2021-10-28T02:24:01Z,...,Pop,https://api.soundcloud.com/playlists/1052484982,517254165,NaN,False,2020-05-16T20:37:26Z,2020-05-16T20:37:26Z,33,/pop-smoke-12345/sets/pop-smoke,"656255261,610276575,752519617,687445048,752519..."
4,NaN,2018-06-07T22:39:43Z,NaN,231259392,all,Electronic,536039613,playlist,NaN,2018-10-01T14:49:34Z,...,Electronic,https://api.soundcloud.com/playlists/536039613,403437198,NaN,False,2018-06-12T00:16:07Z,2018-06-12T00:16:07Z,476,/user-733308340-928877766/sets/electronic-scamp,"304971267,124259977,168029313,174500966,107547..."


### 1.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_pl` để chứa các cột cần xóa

In [230]:
del_col_pl = []
data = {}

for col in playlists:
    percent = playlists[col].isna().sum() / len(playlists)
    if percent > 0.4:
        data[col] = percent * 100
        del_col_pl.append(col)
        
# Xóa các cột ra khỏi data
playlists = playlists.drop(del_col_pl, axis=1)

# In ra các cột bị xóa đi và phần trăm của chúng
table = pd.DataFrame(data)
table.index = ["Phần trăm (%)"]
table.T

Index(['created_at', 'duration', 'embeddable_by', 'genre', 'id', 'kind',
       'last_modified', 'license', 'likes_count', 'managed_by_feeds',
       'permalink', 'permalink_url', 'public', 'reposts_count', 'sharing',
       'tag_list', 'title', 'uri', 'user_id', 'is_album', 'display_date',
       'track_count', 'url', 'TracksId'],
      dtype='object')

- Xem lại thông tin của `playlists` sau khi đã loại bỏ các cột không cần thiết

In [214]:
print(f'Số dòng và số cột dữ liệu của file playlist.csv {playlists.shape}')
print(f'Số dòng dữ liệu {playlists.shape[0]}')
print(f'Số cột dữ liệu {playlists.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
playlists.head()

Số dòng và số cột dữ liệu của file playlist.csv (1490, 24)
Số dòng dữ liệu 1490
Số cột dữ liệu 24


,created_at,duration,embeddable_by,genre,id,kind,last_modified,license,likes_count,managed_by_feeds,...,sharing,tag_list,title,uri,user_id,is_album,display_date,track_count,url,TracksId
0,2017-09-15T02:41:19Z,114356108,all,Reggae,352781337,playlist,2020-04-04T16:19:58Z,all-rights-reserved,1,False,...,public,Dancehall,Dancehall,https://api.soundcloud.com/playlists/352781337,197755511,False,2017-09-15T02:41:19Z,448,/user-818098543/sets/dancehall,"5963010,163143507,176991526,115524663,17063332..."
1,2019-08-09T15:27:57Z,10917613,all,Chill EDM,843427727,playlist,2021-11-12T14:36:50Z,all-rights-reserved,10445,False,...,public,"Waves ThePeak Chill ""EDM Chill"" Electronic Bas...",Chill EDM: Waves,https://api.soundcloud.com/playlists/843427727,603452106,False,2019-08-09T20:00:44Z,53,/soundcloud-the-peak/sets/waves-chill-edm,"1153337647,1142050318,1138569328,1146058903,11..."
2,2014-12-31T06:29:43Z,150327799,none,Electronic,68396635,playlist,2017-01-20T03:05:06Z,all-rights-reserved,2,False,...,public,"Hardstyle Dubstep ""Electro House"" ""Glitch Hop""...",Dubstep,https://api.soundcloud.com/playlists/68396635,109662100,False,2014-12-31T06:29:43Z,456,/damogaur86/sets/dubstep,"130054263,154791745,171835907,26549785,1608792..."
3,2020-05-16T20:37:26Z,5554371,all,Hip-hop & Rap,1052484982,playlist,2021-10-28T02:24:01Z,all-rights-reserved,9036,False,...,public,"""Pop Smoke"" Pop Smoke",Pop,https://api.soundcloud.com/playlists/1052484982,517254165,False,2020-05-16T20:37:26Z,33,/pop-smoke-12345/sets/pop-smoke,"656255261,610276575,752519617,687445048,752519..."
4,2018-06-07T22:39:43Z,231259392,all,Electronic,536039613,playlist,2018-10-01T14:49:34Z,all-rights-reserved,1,False,...,public,NaN,Electronic,https://api.soundcloud.com/playlists/536039613,403437198,False,2018-06-12T00:16:07Z,476,/user-733308340-928877766/sets/electronic-scamp,"304971267,124259977,168029313,174500966,107547..."


### **1.2) Thông tin về các cột dữ liệu còn lại của `playlists`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**created_at**|Thời gian|Thời gian khởi tạo playlist|
|**duration**|Số nguyên|Tổng thời gian các track nhạc trong playlist|
|**embeddable_by**|Chuỗi|
|**genre**|Chuỗi|Thể loại nhạc của playlist|
|**id**|Kiểu phân loại|Định danh cho playlist|
|**kind**|Chuỗi|Loại user|  
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của tác giả| 
|**license**|Chuỗi|Giấy phép|
|**likes_count**|Số nguyên|Số lượt yêu thích của playlist|
|**managed_by_feeds**|Luận lý|Playlist có được quản lý bởi Feed|
|**permalink**|Chuỗi|  
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |  
|**public**|Luận lý|Playlist có công khai hay không?|   
|**reposts_count**|Số nguyên|Số lượt đăng lại playlist|
|**sharing**|Chuỗi|Chế độ chia sẻ công khai hay riêng tư|
|**tag_list**|Chuỗi|
|**title**|Chuỗi|Tiêu đề playlist|    
|**uri**|Chuỗi|Đường dẫn API của playlist|    
|**user_id**|Kiểu phân loại|Định danh của người sở hữu playlist|
|**is_album**|Luận lý|Playlist có phải album hay không|
|**display_date**|Thời gian|Thời điểm playlist hiển thị|   
|**track_counts**|Số nguyên|Số lượng track trong playlist|
|**url**|Chuỗi|Đường dẫn API của playlist|   
|**TracksId**|Chuỗi|Danh sách id của các track trong playlist được ngăn cách bởi ","|

---

### **2) Phân tích đối với tập user:**

Sử dụng lại biến `users` (Data Frame) đã khai báo ở B7.

In [216]:
print(f'Số dòng và số cột dữ liệu của file user.csv {users.shape}')
print(f'Số dòng dữ liệu {users.shape[0]}')
print(f'Số cột dữ liệu {users.shape[1]}')

Số dòng và số cột dữ liệu của file user.csv (1354, 39)
Số dòng dữ liệu 1354
Số cột dữ liệu 39


In [217]:
# Xem thông tin 5 dòng đầu tiên
users.head()

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,first_name,...,station_permalink,url,creator_subscription.product.id,badges.pro,badges.pro_unlimited,badges.verified,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,https://i1.sndcdn.com/avatars-000055961532-om0...,NaN,86,NaN,2013-07-29T05:25:14Z,[{'product': {'id': 'free'}}],NaN,1411,1741,ʚïɞSue,...,artist-stations:52557367,/saratology,free,False,False,False,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/avatars-000435801174-dzp...,Los Angeles,0,US,2013-03-05T17:51:51Z,[{'product': {'id': 'creator-pro-unlimited'}}],NaN,4964,2,NaN,...,artist-stations:38296485,/backlotmusic,creator-pro-unlimited,False,True,False,soundcloud:users:38296485,True,"[{'urn': 'soundcloud:visuals:111760284', 'entr...",NaN
2,https://i1.sndcdn.com/avatars-IMLvkSzRUua5FWaU...,NaN,2,NaN,2018-04-16T16:18:54Z,[{'product': {'id': 'free'}}],NaN,8,40,SATIVA,...,artist-stations:434139399,/sativa-mitchell-21088,free,False,False,False,soundcloud:users:434139399,True,"[{'urn': 'soundcloud:visuals:46593301', 'entry...",NaN
3,https://i1.sndcdn.com/avatars-000220689128-a2w...,NaN,6,NaN,2013-09-09T03:12:12Z,[{'product': {'id': 'free'}}],NaN,81,119,NaN,...,artist-stations:57382585,/carocj,free,False,False,False,soundcloud:users:57382585,True,"[{'urn': 'soundcloud:visuals:9268568', 'entry_...",NaN
4,https://i1.sndcdn.com/avatars-jTAoMxJjdFXggopi...,reading studying zen,0,NaN,2020-04-13T13:57:47Z,[{'product': {'id': 'free'}}],NaN,785,9,for meditation epic chill workout,...,artist-stations:809060917,/music-for-life-playlists,free,False,False,False,NaN,NaN,NaN,NaN


### 2.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_us` để chứa các cột cần xóa

In [226]:
del_col_us = []
data = {}

for col in users:
    percent = users[col].isna().sum() / len(users)
    if percent > 0.4:
        data[col] = percent * 100
        del_col_us.append(col)
        
# Xóa các cột ra khỏi data
users = users.drop(del_col_us, axis=1)

# In ra các cột bị xóa đi và phần trăm của chúng
table = pd.DataFrame(data)
table.index = ["Phần trăm (%)"]
table.T

cột city không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột country_code không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột description không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột last_name không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột reposts_count không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột visuals không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột visuals.tracking không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi


Index(['avatar_url', 'comments_count', 'created_at', 'creator_subscriptions',
       'followers_count', 'followings_count', 'first_name', 'full_name',
       'groups_count', 'id', 'kind', 'last_modified', 'likes_count',
       'playlist_likes_count', 'permalink', 'permalink_url', 'playlist_count',
       'track_count', 'uri', 'urn', 'username', 'verified', 'station_urn',
       'station_permalink', 'url', 'creator_subscription.product.id',
       'badges.pro', 'badges.pro_unlimited', 'badges.verified', 'visuals.urn',
       'visuals.enabled', 'visuals.visuals'],
      dtype='object')

- Xem lại thông tin của `users` sau khi đã loại bỏ các cột không cần thiết

In [231]:
print(f'Số dòng và số cột dữ liệu của file user.csv {users.shape}')
print(f'Số dòng dữ liệu {users.shape[0]}')
print(f'Số cột dữ liệu {users.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
users.head()

Số dòng và số cột dữ liệu của file user.csv (1354, 32)
Số dòng dữ liệu 1354
Số cột dữ liệu 32


,avatar_url,comments_count,created_at,creator_subscriptions,followers_count,followings_count,first_name,full_name,groups_count,id,...,station_urn,station_permalink,url,creator_subscription.product.id,badges.pro,badges.pro_unlimited,badges.verified,visuals.urn,visuals.enabled,visuals.visuals
0,https://i1.sndcdn.com/avatars-000055961532-om0...,86,2013-07-29T05:25:14Z,[{'product': {'id': 'free'}}],1411,1741,ʚïɞSue,ʚïɞSue Catyʚïɞ,0,52557367,...,soundcloud:system-playlists:artist-stations:52...,artist-stations:52557367,/saratology,free,False,False,False,NaN,NaN,NaN
1,https://i1.sndcdn.com/avatars-000435801174-dzp...,0,2013-03-05T17:51:51Z,[{'product': {'id': 'creator-pro-unlimited'}}],4964,2,NaN,NaN,0,38296485,...,soundcloud:system-playlists:artist-stations:38...,artist-stations:38296485,/backlotmusic,creator-pro-unlimited,False,True,False,soundcloud:users:38296485,True,"[{'urn': 'soundcloud:visuals:111760284', 'entr..."
2,https://i1.sndcdn.com/avatars-IMLvkSzRUua5FWaU...,2,2018-04-16T16:18:54Z,[{'product': {'id': 'free'}}],8,40,SATIVA,SATIVA MITCHELL.21088,0,434139399,...,soundcloud:system-playlists:artist-stations:43...,artist-stations:434139399,/sativa-mitchell-21088,free,False,False,False,soundcloud:users:434139399,True,"[{'urn': 'soundcloud:visuals:46593301', 'entry..."
3,https://i1.sndcdn.com/avatars-000220689128-a2w...,6,2013-09-09T03:12:12Z,[{'product': {'id': 'free'}}],81,119,NaN,NaN,0,57382585,...,soundcloud:system-playlists:artist-stations:57...,artist-stations:57382585,/carocj,free,False,False,False,soundcloud:users:57382585,True,"[{'urn': 'soundcloud:visuals:9268568', 'entry_..."
4,https://i1.sndcdn.com/avatars-jTAoMxJjdFXggopi...,0,2020-04-13T13:57:47Z,[{'product': {'id': 'free'}}],785,9,for meditation epic chill workout,for meditation epic chill workout sad relaxing...,0,809060917,...,soundcloud:system-playlists:artist-stations:80...,artist-stations:809060917,/music-for-life-playlists,free,False,False,False,NaN,NaN,NaN


### **2.2) Thông tin về các cột dữ liệu còn lại của `users`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa  
|:--------:|:-------------:|:-----------:
|**avatar_url**|Chuỗi |Đường dẫn đến ản đại diện của user|
|**created_at**|Thời gian|Thời gian khởi tạo user|
|**id**|Kiểu phân loại|Định danh của user|
|**comments_count**|Số nguyên|Số lượt bình luận của user|
|**likes_count**|Số nguyên|Số lượt được yêu thích của user|
|**followers_count**|Số nguyên|Số lượt theo dõi của user|
|**followings_count**|Số nguyên|Số lượt user theo dõi|
|**playlist_likes_count**|Số nguyên|Số lượt được yêu thích các playlist của user|
|**playlist_count**|Số nguyên|Số playlist của user|
|**track_count**|Số nguyên|Số lượng track của user|
|**kind**|Chuỗi|Loại đối tượng|
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng| 
|**first_name**|Chuỗi|Tên của user|  
|**full_name**|Chuỗi|Tên đầy đủ của user| 
|**username**|Chuỗi|Tên đăng nhập của user|   
|**groups_count**|Số nguyên|Số nhóm của user|
|**permalink**|Chuỗi|Đường dẫn cố định của tài nguyên| 
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |   
|**uri**|Chuỗi|Đường dẫn API của user|    
|**url**|Chuỗi|Đường dẫn API của user|    
|**urn**|Chuỗi|Đường dẫn API của user|    
|**creator_subscriptions**|Chuỗi|Loại thành viên của user|    
|**creator_subscription.product.id**|Chuỗi|Loại thành viên của user|    
|**badges.pro_unlimited'**|Luận lý|User có phải người dùng pro_unlimited không|
|**verified**|Luận lý|Xác thực người dùng|
|**visuals.urn**|Chuỗi
|**visuals.enabled**|Chuỗi|
|**visuals.visuals**|Chuỗi|

---

### **3) Phân tích đối với tập track:**

Sử dụng lại biến `tracks` (Data Frame) đã khai báo ở B8.

In [232]:
print(f'Số dòng và số cột dữ liệu của file track.csv {tracks.shape}')
print(f'Số dòng dữ liệu {tracks.shape[0]}')
print(f'Số cột dữ liệu {tracks.shape[1]}')

Số dòng và số cột dữ liệu của file track.csv (2863, 67)
Số dòng dữ liệu 2863
Số cột dữ liệu 67


In [234]:
# Xem thông tin 5 dòng đầu tiên
tracks.head()

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,...,publisher_metadata.c_line,publisher_metadata.c_line_for_display,publisher_metadata.release_title,publisher_metadata.writer_composer,publisher_metadata.publisher,publisher_metadata.iswc,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,NaN,NaN,True,1.0,2012-11-27T19:02:04Z,NaN,False,0.0,420453,420453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000045729697-23...,NaN,True,33.0,2013-04-17T13:32:12Z,Artist: Vedomir (Marcel Dettmann remixes)\r\nF...,False,0.0,248943,248943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://i1.sndcdn.com/artworks-000149186638-zs...,NaN,True,35.0,2016-03-01T05:13:23Z,Fresh off of the buzz from his most recent re...,False,0.0,189509,189509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://i1.sndcdn.com/artworks-KOQjX4KNCzmQ-0-...,NaN,True,10.0,2016-04-01T19:36:01Z,NaN,False,0.0,233430,233430,...,"Â© 2006 Gasoline Alley, J.V.","Â© 2006 Gasoline Alley, J.V.","April 29, 1992 (Miami)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://i1.sndcdn.com/artworks-vA1Gvw7Rqbdi-0-...,NaN,True,5.0,2021-11-03T23:36:01Z,NaN,False,0.0,30000,250331,...,"An Atlantic Records UK release, Â© 2021 Warner...","Â© An Atlantic Records UK release, Â© 2021 War...",Faisal (Envelops Me),"Benjy Gibson, Emmanuel Franklyn Adelabu, Fasia...",NaN,NaN,NaN,NaN,NaN,NaN


#### 3.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_tr` để chứa các cột cần xóa

In [235]:
del_col_tr = []
data = {}

for col in tracks:
    percent = tracks[col].isna().sum() / len(tracks)
    if percent > 0.4:
        data[col] = percent * 100
        del_col_tr.append(col)
        
# Xóa các cột ra khỏi data
tracks = tracks.drop(del_col_tr, axis=1)

# In ra các cột bị xóa đi và phần trăm của chúng
table = pd.DataFrame(data)
table.index = ["Phần trăm (%)"]
table.T

cột caption không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột label_name không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột purchase_title không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột purchase_url không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột release_date không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột secret_token không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột visuals không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.id không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.urn không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.artist không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.album_title không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.contains_music không đóng góp nhiều vào dữ liệu do đó sẽ bị xóa đi
cột publisher_metadata.upc_or_ean kh

Index(['artwork_url', 'commentable', 'comment_count', 'created_at',
       'description', 'downloadable', 'download_count', 'duration',
       'full_duration', 'embeddable_by', 'genre', 'has_downloads_left', 'id',
       'kind', 'last_modified', 'license', 'likes_count', 'permalink',
       'permalink_url', 'playback_count', 'public', 'reposts_count', 'sharing',
       'state', 'streamable', 'tag_list', 'title', 'track_format', 'uri',
       'urn', 'user_id', 'waveform_url', 'display_date', 'station_urn',
       'station_permalink', 'track_authorization', 'monetization_model',
       'policy', 'media.transcodings'],
      dtype='object')

- Xem lại thông tin của `tracks` sau khi đã loại bỏ các cột không cần thiết

In [236]:
print(f'Số dòng và số cột dữ liệu của file user.csv {tracks.shape}')
print(f'Số dòng dữ liệu {tracks.shape[0]}')
print(f'Số cột dữ liệu {tracks.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
tracks.head()

Số dòng và số cột dữ liệu của file user.csv (2863, 39)
Số dòng dữ liệu 2863
Số cột dữ liệu 39


,artwork_url,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,embeddable_by,...,urn,user_id,waveform_url,display_date,station_urn,station_permalink,track_authorization,monetization_model,policy,media.transcodings
0,NaN,True,1.0,2012-11-27T19:02:04Z,NaN,False,0.0,420453,420453,all,...,soundcloud:tracks:69064306,29224349,https://wave.sndcdn.com/6mP4lCGoGWlO_m.json,2012-11-27T19:02:04Z,soundcloud:system-playlists:track-stations:690...,track-stations:69064306,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,[{'url': 'https://api-v2.soundcloud.com/media/...
1,https://i1.sndcdn.com/artworks-000045729697-23...,True,33.0,2013-04-17T13:32:12Z,Artist: Vedomir (Marcel Dettmann remixes)\r\nF...,False,0.0,248943,248943,all,...,soundcloud:tracks:88270512,97164,https://wave.sndcdn.com/Jek2uMvy0KX3_m.json,2013-04-17T13:32:12Z,soundcloud:system-playlists:track-stations:882...,track-stations:88270512,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,[{'url': 'https://api-v2.soundcloud.com/media/...
2,https://i1.sndcdn.com/artworks-000149186638-zs...,True,35.0,2016-03-01T05:13:23Z,Fresh off of the buzz from his most recent re...,False,0.0,189509,189509,all,...,soundcloud:tracks:249605409,94400345,https://wave.sndcdn.com/Axw8Czs8OHZT_m.json,2016-03-01T05:13:23Z,soundcloud:system-playlists:track-stations:249...,track-stations:249605409,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,[{'url': 'https://api-v2.soundcloud.com/media/...
3,https://i1.sndcdn.com/artworks-KOQjX4KNCzmQ-0-...,True,10.0,2016-04-01T19:36:01Z,NaN,False,0.0,233430,233430,all,...,soundcloud:tracks:256800168,188512,https://wave.sndcdn.com/YDFdEeWUd1HN_m.json,1996-07-30T00:00:00Z,soundcloud:system-playlists:track-stations:256...,track-stations:256800168,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,BLOCK,[]
4,https://i1.sndcdn.com/artworks-vA1Gvw7Rqbdi-0-...,True,5.0,2021-11-03T23:36:01Z,NaN,False,0.0,30000,250331,all,...,soundcloud:tracks:1153449859,637643742,https://wave.sndcdn.com/WvfeReNh8Nvb_m.json,2021-11-12T00:00:00Z,soundcloud:system-playlists:track-stations:115...,track-stations:1153449859,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,SNIP,[{'url': 'https://api-v2.soundcloud.com/media/...


#### **3.2) Thông tin về các cột dữ liệu còn lại của `tracks`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**artwork_url**|chuỗi |URL ảnh của track|
|**comment_count**|Số nguyên |Số bình luận của một track|
|**created_at**|Thời gian |Thời gian khởi tạo của track|
|**description**|Chuỗi |Mô tả của track đó do user đăng tải|
|**downloadable**|Luận lý |Track có tải về được không?|
|**download_count**|Số nguyên |Số lượt tải về của track|
|**duration**|Số nguyên |Thời gian phát của track|
|**full_duration**|Số nguyên|Thời gian phát của track|
|**embeddable_by**|Chuỗi|
|**genre**|Chuỗi|Thể loại của track|
|**has_downloads_left**|Luận lý|
|**id**|Kiểu phân loại|Định danh cho track|
|**kind**|Chuỗi|Loại track
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của track|
|**license likes_count**|Số nguyên|Số lượt like có bản giấy phép của track|
|**public**|Luận lý|Có công khai trên soundcloud hay không|
|**permalink**|Chuỗi|
|**permalink_url**|Chuỗi|Đường dẫn dến trang soundcloud|
|**playback_count**|Số nguyên|Số lượt nghe của track|
|**public**|Luận lý|Track có được công khai hay không?|
|**reposts_count**|Số nguyên|Số lượt đăng lại của track|
|**sharing**|Chuỗi|Share đường dẫn của track|
|**state**|Chuỗi|Nút trạng thái share đường dẫn của track|
|**streamable**|Luận lý|Track có được trực tuyến hay không?|
|**tag_list**|Chuỗi|Các danh sách track được gợi tag|
|**title**|Chuỗi|Tiêu đề của track|
|**track_format**|Chuỗi|Định dạng của track|
|**uri**|Chuỗi|Đường dẫn API của track
|**urn**|Chuỗi|Đường dẫn API của track
|**user_id**|Chuỗi|Người chủ của track
|**waveform_url**|Chuỗi|
|**display_date**|Thời gian|Thời gian đăng tải track trên track page của user|
|**station_urn**|Chuỗi|
|**station_permalink**|Chuỗi|
|**track_authorization**|Chuỗi|Bản quyền của track|
|**monetization_model**|Chuỗi|
|**policy**|Chuỗi|Điều khoản, chính sách của track|
|**media.transcodings**|Chuỗi|